In [6]:

import pandas as pd
import numpy as np
import glob, os
import json
import copy
import math
from pandas.io.parsers import read_csv

In [10]:
# add utility metrics to final results to ATM mapper
# atm_csv = read_csv("/Users/yixue/Documents/Research/FrUITeR/Results/ATM/atm.csv")
# atm_csv = pd.concat([atm_csv, atm_csv.apply(count_event_num_atm, axis=1).apply(pd.Series)], axis=1)
# atm_csv = pd.concat([atm_csv, atm_csv.apply(calculate_utility_atm, axis=1).apply(pd.Series)], axis=1)
# atm_csv
# atm_csv.to_csv("/Users/yixue/Documents/Research/FrUITeR/Results/atm_shopping_final.csv")

# add utility metrics to final results of other mappers
other_csv = read_csv("/Users/yixue/Documents/Research/FrUITeR/Results/framework_results.csv")
other_csv = pd.concat([other_csv, other_csv.apply(count_event_num_other, axis=1).apply(pd.Series)], axis=1)
other_csv = pd.concat([other_csv, other_csv.apply(calculate_utility_other, axis=1).apply(pd.Series)], axis=1)
# other_csv
other_csv.to_csv("/Users/yixue/Documents/Research/FrUITeR/Results/other_shopping_final.csv")



In [7]:
def clean_naive(data):
    data = data.set_index("gui_mapper")
    data = data.drop("naive0", axis=0)
    data = data.drop("naive1", axis=0)
    data = data.drop("naive2", axis=0)
    data = data.drop("naive3", axis=0)
    data = data.drop("naive4", axis=0)
    data = data.drop("naive5", axis=0)
    data = data.drop("naive6", axis=0)
    data = data.drop("naive7", axis=0)
    data = data.drop("naive8", axis=0)
    return data

def count_event_num_other(test):
    num_events = {}
    num_events['num_src'] = len(eval(test['src_events']))
    transferred_json = json.loads(test['transferred'])
    count = 0
    for trans in transferred_json:
        if trans['id_or_xpath'] != 'NONE':
            count += 1
    num_events['num_trans'] = count
    if pd.isnull(test['gt_events']):
        num_events['num_gt'] = np.NaN
    else:
        num_events['num_gt'] = len(eval(test['gt_events']))
    return num_events

def count_event_num_atm(test):
    num_events = {}
    num_events['num_src'] = len(eval(test['src_events']))
    transferred_json = json.loads(test['transferred'])
    count = 0
    for trans in transferred_json:
        if trans != {}:
            count += 1
    num_events['num_trans'] = count
    num_events['num_gt'] = len(eval(test['gt_events']))
    return num_events

def calculate_utility_atm(test):
    result = {}
    if np.isnan(test['num_gt']):
        result['reduction'] = np.NaN
        return result
    try:
        result['reduction'] = (test['num_gt'] - test['distance']) / test['num_gt']
    except ZeroDivisionError:
        result['reduction'] = np.NaN
    return result

def calculate_utility_other(test):
    result = {}
    try:
        result['reduction'] = (test['num_gt'] - test['distance']) / test['num_gt']
    except ZeroDivisionError:
        result['reduction'] = np.NaN
    return result

In [11]:
# merge atm's results and others 
atm_csv = read_csv("/Users/yixue/Documents/Research/FrUITeR/Results/atm_shopping_final.csv")
other_csv = read_csv("/Users/yixue/Documents/Research/FrUITeR/Results/other_shopping_final.csv") 
merged_csv = atm_csv.append(other_csv)
# merged_csv
merged_csv.to_csv("/Users/yixue/Documents/Research/FrUITeR/Results/shopping_all.csv")
